In [1]:
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U accelerate jupyter ipywidgets
!pip install gymnasium
import gymnasium as gym
import math
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    GenerationConfig,
    pipeline,
)
import torch
from transformers import pipeline
import csv
import datetime

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.4/123.4 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.5/93.5 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 10.8 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
resultsFile='/content/drive/MyDrive/Results/CartPoleLlm.csv'
overWriteResultsFile = True

In [4]:
job_description = """You are a large language model playing a game called CartPole. Respond with 'Left' or 'Right' without adding comments or explanations."""

In [5]:
zero_shot_prompt = """In which direction the cart of a cart pole should be moved if the pole direction is #degrees degrees
off to the #direction?
Respond with 'Left' or 'Right'. Do not add comments."""

one_shot_prompt = """When the pole tilts to the right the cart should be moved to the right.
In which direction the cart should be moved if the pole direction is #degrees degrees off to the #direction?
Respond with 'Left' or 'Right'. Do not add comments."""

few_shot_prompt = """When the pole tilts to the right the cart should be moved to the right.
When the pole tilts to the left the cart should be moved to the left.
In which direction the cart should be moved if the pole direction is #degrees degrees off to the #direction?
Respond with 'Left' or 'Right'. Do not add comments."""

Chain_of_Thought_prompt = """Q: In which direction the cart of a cart pole should be moved if
the pole direction is 5 degrees off to the left
A: Left
Q: In which direction the cart of a cart pole should be moved if
the pole direction is 5 degrees off to the right
A: Right
Q: In which direction the cart of a cart pole should be moved if
the pole direction is #degrees degrees off to the #direction?"""


prompts = [zero_shot_prompt,one_shot_prompt,few_shot_prompt,Chain_of_Thought_prompt]
#prompts = [Chain_of_Thought_prompt]
prompts = {
    "zero_shot_prompt": zero_shot_prompt,
    "one_shot_prompt": one_shot_prompt,
    "few_shot_prompt": few_shot_prompt,
    "Chain_of_Thought_prompt": Chain_of_Thought_prompt
}

In [6]:
temperatures = [0.1, 0.4, 0.7, 1]

In [7]:
models_dict = {
    "StableLM Zephyr 3B": "stabilityai/stablelm-zephyr-3b"
}

model_id = models_dict["StableLM Zephyr 3B"]

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_id)#, device=0)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/5.21k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/587 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
# Running this may take a minute
pipe = pipeline("text-generation", model=model_id, device_map="auto", tokenizer=tokenizer, torch_dtype=torch.bfloat16)

config.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/5.59G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [10]:
def generate(question, temperature=0.7, max_new_tokens=512):
    #return "left"
    messages = [
        {
            "role": "system",
            "content": job_description,
        },
    ]
    messages.append({"role": "user", "content": question})
    prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    outputs = pipe(prompt, max_new_tokens=max_new_tokens, do_sample=True, temperature=temperature, top_k=50, top_p=0.95)
    output = outputs[0]["generated_text"]
    messages.append({"role": "assistant", "content": output})
    #print(output)
    response_start = output.rfind('<|assistant|>')
    text_output = output[response_start + len('<|assistant|>'):]
    return text_output

In [11]:
def last_word_used(text):
    # Find the last index of "left" and "right"
    left_index = text.rfind("left")
    right_index = text.rfind("right")

    # Compare the indices to determine the last word used
    if left_index > right_index:
        return "left"
    elif right_index > left_index:
        return "right"
    else:
        # Both words have the same index, handle this case as needed
        return "Both words appear at the same position"

In [12]:
def insertValuesInPrompt(prompt, degrees, direction):
    prompt = prompt.replace("#degrees", str(degrees))
    prompt = prompt.replace("#direction", direction)
    #print(prompt)
    return prompt

In [13]:
#Create csv file
def createNewResultsCsv():
    if(overWriteResultsFile):
        with open(resultsFile, 'w', newline='') as csvfile:
            # Create a CSV writer object
            csvwriter = csv.writer(csvfile)
            csvwriter.writerow(["promptKey", "temperature", "timesteps", "rewards", "leftActions", "rightActions", "randomActions", "DateTime"])

In [14]:
def runExperiment(episodes,promptKey, temperature):
    env = gym.make('CartPole-v1')#, render_mode="human")
    if(promptKey=="Random"):
      prompt = "Random"
    else:
      prompt = prompts[promptKey]

    with open(resultsFile, 'a', newline='') as csvfile:
        # Create a CSV writer object
        csvwriter = csv.writer(csvfile)
        for i_episode in range(episodes):
            observation = env.reset()
            rad = observation[0][2]
            rewards = 0
            randomActions = 0
            leftActions = 0
            rightActions = 0

            for t in range(250):
                env.render()

                #action = env.action_space.sample()

                degrees = round(math.degrees(rad), 2)
                direction = "right"
                if degrees < 0:
                    direction = "left"
                    degrees = degrees * -1

                if(prompt=="Random"):
                  output = "Random"
                else:
                  output = generate(insertValuesInPrompt(prompt, degrees, direction), temperature, 15)

                action = 0
                #print("OUTPUT")
                #print(output)
                #print("ENDOUTPUT")
                if last_word_used(output.lower()) == "right":
                    action = 1
                    actionText = 'right'
                    rightActions+=1
                elif last_word_used(output.lower()) == "left":
                    action = 0
                    actionText = 'left'
                    leftActions+=1
                else:
                    actionText = 'random'
                    randomActions+=1
                    action = env.action_space.sample() #Inconclusive answer, random action
                    if(action==0):
                      leftActions+=1
                    else:
                      rightActions+=1
                #print(output)
                #print(str(degrees) + " " + direction + " answer: "+actionText+" action: " + str(action))
                observation, reward, terminated, truncated, info = env.step(action)
                rad = observation[2]
                rewards += reward

                if terminated:
                    print('prompt: {}, temperature:{}, timesteps: {}, total rewards: {}, left: {}, right: {}, random: {}, datetime'.format(promptKey, temperature, t+1, rewards,leftActions,rightActions,randomActions,datetime.datetime.now()))
                    if(overWriteResultsFile):
                        csvwriter.writerow([promptKey, temperature, t+1, rewards, leftActions, rightActions, randomActions,datetime.datetime.now()])
                    break

        env.close()


In [15]:
def runExperiments(episodes,prompts, temperatures):
  createNewResultsCsv()
  for promptKey in prompts.keys():
    for temperature in temperatures:
      runExperiment(episodes, promptKey, temperature)

In [16]:
#Experiments in multiple sessions because of runtime limitations.
resultsFile='/content/drive/MyDrive/Results/CartPoleLlm_'+'Total'+'_'+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")+'.csv'
prompts = {
    "zero_shot_prompt": zero_shot_prompt,
    "one_shot_prompt": one_shot_prompt,
    "few_shot_prompt": few_shot_prompt,
    "Chain_of_Thought_prompt": Chain_of_Thought_prompt
}

In [17]:
runExperiments(100,prompts, temperatures)
runExperiment(100, "Random", 0)

/usr/local/lib/python3.10/dist-packages/gymnasium/envs/classic_control/cartpole.py:215: UserWarning: WARN: You are calling render method without specifying any render mode. You can specify the render_mode at initialization, e.g. gym.make("CartPole-v1", render_mode="rgb_array")
  gym.logger.warn(
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


prompt: zero_shot_prompt, temperature:0.1, timesteps: 9, total rewards: 9.0, left: 9, right: 0, random: 0, datetime
prompt: zero_shot_prompt, temperature:0.1, timesteps: 9, total rewards: 9.0, left: 9, right: 0, random: 0, datetime
prompt: zero_shot_prompt, temperature:0.1, timesteps: 10, total rewards: 10.0, left: 10, right: 0, random: 0, datetime
prompt: zero_shot_prompt, temperature:0.1, timesteps: 10, total rewards: 10.0, left: 9, right: 1, random: 1, datetime
prompt: zero_shot_prompt, temperature:0.1, timesteps: 9, total rewards: 9.0, left: 9, right: 0, random: 0, datetime
prompt: zero_shot_prompt, temperature:0.1, timesteps: 10, total rewards: 10.0, left: 10, right: 0, random: 0, datetime
prompt: zero_shot_prompt, temperature:0.1, timesteps: 10, total rewards: 10.0, left: 10, right: 0, random: 0, datetime
prompt: zero_shot_prompt, temperature:0.1, timesteps: 8, total rewards: 8.0, left: 8, right: 0, random: 0, datetime
prompt: zero_shot_prompt, temperature:0.1, timesteps: 10, tot

In [18]:
#generate("In which direction the cart of a cart pole should be moved if the pole direction is 3 degrees off to the right? Respond with 'Left' or 'Right'. Do not add comments.",0.1,150)